In [3]:
import pandas as pd
df = pd.read_excel('AllData_2023-05-05_collated.xlsx')
df.sort_values(by=['Date','Timestamp'], ascending=[True,True], inplace=True)
df.head()

,Date,Timestamp,HDR_CDW_FL,HDR_CHW_FL,HDR_CDWRT,HDR_CDWST,HDR_CHWRT,HDR_CHWST,HDR2_CDW_FL,HDR2_CHW_FL,HDR2_CDWRT,HDR2_CDWST,HDR2_CHWRT,HDR2_CHWST
27597,03-23-2023,10:41,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
27647,03-23-2023,10:42,76.183693,70.183693,35.183689,30.183689,15.18369,10.18369,76.183693,70.183693,35.183689,30.183689,15.18369,10.18369
27657,03-23-2023,10:43,76.183693,70.183693,35.183689,30.183689,15.18369,10.18369,NaN,NaN,NaN,NaN,NaN,NaN
27736,03-23-2023,10:44,76.183693,70.183693,35.183689,30.183689,15.18369,10.18369,76.183693,70.183693,35.183689,30.183689,15.18369,10.18369
27772,03-23-2023,10:45,76.183693,70.183693,35.183689,30.183689,15.18369,10.18369,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
import os
import pandas as pd
import numpy as np
files_with_errors = []
directory = "C:\\Users\\tengwei.goh\\Desktop\\Sample Data\\test\\e2i_test\\missing data"
os.chdir(directory)
collated_df = pd.DataFrame(columns=['Date','Timestamp'])
collated_df = collated_df.set_index(['Date','Timestamp'])
# temp_df = pd.DataFrame()
count = 0
cols_to_use = ['Date', 'Timestamp']
# collated_df = pd.DataFrame()
for root,dirs,files in os.walk(directory):
    for file in files:
        count += 1
        # print(file)
        # Ignore collated file to avoid errors
        if file.endswith('collated.csv'):
            continue

        if file.endswith('.csv'):
            # try:
                f = open(file, 'r')
                result = []
                for l in f.readlines():
                    vals = [l for l in l.split('#') if l]
                    index = vals[0]
                    result.append(index)
                ...
                # fill dataframe
                f.close()

                for i in range(len(result)):
                    result[i] = result[i].replace('\n','')
                    # result = result.split(',')
                    # print(result[i])

                temp = result[1:]
                # print(temp[0])
                for i in range(len(temp)):
                    temp[i] = temp[i].split(',')
                df = pd.DataFrame(temp[1:],columns=temp[0])
                try:
                    df.columns.get_loc('Timestamp')
                except:
                    df = df.rename(columns={'Time': 'Timestamp'})
                df = df.replace(r'^\s*$', np.nan, regex=True)
                df = df[df['Timestamp'].notna()]
                df = df.reset_index(drop=True)
                print(df.head())


                try:
                    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
                except:
                    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
                
                for i in range(len(df)):
                    temp = df['Timestamp'][i].split(':')
                    hour = str('%02d' % int(temp[0])) + ':'
                    minute = str('%02d' % int(temp[1]))
                    df['Timestamp'][i] = hour + minute
                # print(df.head())
                #TODO: FINALLY FIXED
                # Need these columns to keep original column order
                cols = collated_df.columns.append(df.columns).unique()
                cols = cols.drop(['Date','Timestamp'])

                # NEED TO DROP EMPTY COLUMN NAMES AND CELLS to prevent error
                df = df.loc[:, df.columns.notna()]
                df = df[df['Timestamp'].notna()]
                # print(df.head())
                df = insert_empty_slot_1(df)
                # print(df.head())
                df = df.set_index(['Date','Timestamp'])
                # print(df.head())
                collated_df = collated_df.combine_first(df).reindex(columns=cols)
                # return collated_df, files_with_errors
                print(files_with_errors)
                # print(collated_df.head())
                collated_df.to_excel('collated.xlsx')
            # except:
            #     try:
            #         df = pd.read_csv(file)
            #         df = df.reset_index(drop=True)
            #         # f = open(file, 'r')
            #         # result = []
            #         # for l in f.readlines():
            #         #     vals = [l for l in l.split('#') if l]
            #         #     index = vals[0]
            #         #     # print(index)
            #         #     result.append(index)
            #         # ...
            #         # # fill dataframe
            #         # f.close()
            #         # # print(result)
            #         # for i in range(len(result)):
            #         #     result[i] = result[i].replace('\n','')
            #         #     # result = result.split(',')

            #         # cols = result[0].split(',')
            #         # temp = result[1:]
            #         # # print(temp[0])
            #         # for i in range(len(temp)):
            #         #     temp[i] = temp[i].split(',')
            #         # df = pd.DataFrame(temp[1:],columns=cols)
            #         # # print(df.head())
            #         try:
            #             df.columns.get_loc('Timestamp')
            #         except:
            #             df = df.rename(columns={'Time': 'Timestamp'})
            #         try:
            #             df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
            #         except:
            #             df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
            #         # print(df.Date)
            #         # for i in range(len(df)):
            #         #     temp = df['Date'][i].split('/')
            #         #     day = str('%02d' % int(temp[0])) + '/'
            #         #     month = str('%02d' % int(temp[1])) + '/'
            #         #     year = str('%04d' % int(temp[2]))
            #         #     df['Date'][i] = day + month + year
            #         # print(df.Date)
            #         df['Timestamp'] = df['Timestamp'].astype(str)
            #         # print(df['Timestamp'])
            #         for i in range(len(df)):
            #             temp = df['Timestamp'][i].split(':')
            #             hour = str('%02d' % int(temp[0])) + ':'
            #             minute = str('%02d' % int(temp[1]))
            #             df['Timestamp'][i] = hour + minute
            #             # print(df['Timestamp'])

            #         # Need these columns to keep original column order
            #         cols = collated_df.columns.append(df.columns).unique()
            #         cols = cols.drop(['Date','Timestamp'])

            #         # NEED TO DROP EMPTY COLUMN NAMES AND CELLS to prevent error
            #         df = df.loc[:, df.columns.notna()]
            #         df = df[df['Timestamp'].notna()]
            #         df = insert_empty_slot_1(df)
            #         # print(df.Date)
            #         df = df.set_index(['Date','Timestamp'])
            #         # print(df.head())
            #         collated_df = collated_df.combine_first(df).reindex(columns=cols)
            #         # return collated_df, files_with_errors
            #         print(files_with_errors)
            #         # print(collated_df.head())
            #     except Exception as e:
            #         print(e)
            #         files_with_errors.append(file)
            #         # return collated_df, files_with_errors
            #         print(files_with_errors)
            #         # print(collated_df.head())

       Date Timestamp CHW-Chilled Water Supply Temperature  \
0  1/4/2022   0:21:00                                 7.22   
1  1/4/2022   0:22:00                                 7.21   
2  1/4/2022   0:23:00                                 7.21   
3  1/4/2022   0:24:00                                 7.21   
4  1/4/2022   0:25:00                                  7.2   

  CHW-Chilled Water Return Temperature CHW-Chilled Water Flow Rate  \
0                                 8.49                       16.19   
1                                 8.45                          16   
2                                 8.47                       16.15   
3                                 8.47                       16.04   
4                                 8.49                       16.16   

  CNW-Chilled Water Supply Temperature CNW-Chilled Water Return Temperature  \
0                                26.68                                   29   
1                                26.67          

C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_7980\3853361815.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = hour + minute


[]
       Date Timestamp CHW-Chilled Water Supply Temperature  \
0  02/04/22   0:1:00:                                 7.06   
1  02/04/22   0:2:00:                                 7.05   
2  02/04/22   0:3:00:                                 7.04   
3  02/04/22   0:4:00:                                 7.01   
4  02/04/22   0:5:00:                                 7.00   

  CHW-Chilled Water Return Temperature CHW-Chilled Water Flow Rate  \
0                                 8.78                       16.14   
1                                 8.76                       16.20   
2                                 8.75                       15.97   
3                                 8.74                       16.12   
4                                 8.76                       16.05   

  CNW-Chilled Water Supply Temperature CNW-Chilled Water Return Temperature  \
0                                25.98                                29.11   
1                                25.97       

C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_7980\3853361815.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = hour + minute


[]
       Date Timestamp Chiller Q1-Incoming Chiller Q1-1 Chiller Q1-2  \
0  01/04/22   0:1:00:                4.00         0.08         0.00   
1  01/04/22   0:2:00:                4.00         0.08         0.00   
2  01/04/22   0:3:00:                4.00         0.08         0.00   
3  01/04/22   0:4:00:                4.00         0.08         0.00   
4  01/04/22   0:5:00:                4.00         0.08         0.00   

  Chiller Q1-3 Chiller Q1-4 Chiller Q1-5 Chiller Q1-6 Chiller Q1-7  ...  \
0         0.00         1.64         0.08         0.00         0.00  ...   
1         0.00         1.66         0.08         0.00         0.00  ...   
2         0.00         1.64         0.08         0.00         0.00  ...   
3         0.00         1.62         0.08         0.00         0.00  ...   
4         0.00         1.63         0.08         0.00         0.00  ...   

  Chiller Q2-3 Chiller Q2-4 Chiller Q2-5 Chiller Q2-6 Chiller Q2-7  \
0         0.00         0.04         0.08         

C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_7980\3853361815.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = hour + minute


[]
       Date Timestamp Chiller Q1-Incoming Chiller Q1-1 Chiller Q1-2  \
0  2/4/2022   0:21:00                   3         0.08            0   
1  2/4/2022   0:22:00                   3         0.08            0   
2  2/4/2022   0:23:00                   3         0.08            0   
3  2/4/2022   0:24:00                   3         0.08            0   
4  2/4/2022   0:25:00                   3         0.08            0   

  Chiller Q1-3 Chiller Q1-4 Chiller Q1-5 Chiller Q1-6 Chiller Q1-7  ...  \
0            0         1.61         0.08            0            0  ...   
1            0         1.62         0.08            0            0  ...   
2            0         1.61         0.08            0            0  ...   
3            0         1.61         0.08            0            0  ...   
4            0         1.64         0.08            0            0  ...   

  Chiller Q2-3 Chiller Q2-4 Chiller Q2-5 Chiller Q2-6 Chiller Q2-7  \
0            0         0.04         0.08         

C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_7980\3853361815.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = hour + minute


[]


In [93]:
import pandas as pd
import os
directory = "C:\\Users\\tengwei.goh\\Desktop\\Sample Data\\KAL_test"
os.chdir(directory)
vendor = os.path.basename(directory)
files_with_errors = []
collated_df = pd.DataFrame(columns=['Date','Timestamp'])
collated_df = collated_df.set_index(['Date','Timestamp'])
                   
for root,dirs,files in os.walk(directory):
        for file in files:
            if file.endswith('collated.xlsx'):
                continue
                
            if file.endswith('.xlsx'):
                df = pd.read_excel(file, sheet_name=0)
                df = df.reset_index(drop=True)
                df.columns = df.iloc[7-2]
                df = df.drop(df.index[:7-1])
                df = df.reset_index()
                df = df.dropna(how='all')
                df.drop(columns=['index'], axis=1, inplace=True)
                print(file)
                try:
                    df.columns.get_loc('Timestamp')
                except:
                    df = df.rename(columns={'Time': 'Timestamp'})

                df['date and time'] = df['date and time'].str.split(' ')

                for i in range(len(df['date and time'])):
                    try:
                        df['Date'] = df['date and time'][0][0]
                        df['Timestamp'][i] = df['date and time'][i][1][1:6]
                        
                    except:
                        pass
                    
                df.drop(columns=['date and time'], axis=1, inplace=True)
                # NEED TO DROP EMPTY COLUMN NAMES to prevent error
                df = df.loc[:, df.columns.notna()]
                df = df[df['Timestamp'].notna()]
                df = insert_empty_slot_1(df)
                df = df.set_index(['Date','Timestamp'])
                collated_df = collated_df.combine_first(df)
                collated_df = collated_df.rename_axis(None, axis=1)
                # print(collated_df.columns)
                # collated_df = pd.merge(collated_df, df, on=['Date','Timestamp'], how='outer')
    
collated_df.sort_values(by=['Date','Timestamp'], inplace=True, ascending=True)
print(f'These are the files with errors: {files_with_errors}')
# current_date = get_current_date()
collated_df.to_excel('collated.xlsx')

GMreport_20220827-080004.xlsx


C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_6544\2541834201.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = df['date and time'][i][1][1:6]


GMreport_20220828-080004.xlsx


C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_6544\2541834201.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = df['date and time'][i][1][1:6]


These are the files with errors: []


In [1]:
def insert_empty_slot_1(df):
    # df = pd.read_csv('2023-01-30.csv')
    hour_minute_list = [[i,j] for i in range(0,24) for j in range(0,60)]
    for i in range(len(hour_minute_list)):
        hour_minute_list[i] = ['%02d' % hour_minute_list[i][0], '%02d' % hour_minute_list[i][1]]
    # print(hour_minute_list)
    for i in range(len(df['Timestamp'])):
        temp = df['Timestamp'][i].split(':')
        if [temp[0],temp[1]] in hour_minute_list:
            hour_minute_list.remove([temp[0],temp[1]])
    # print(hour_minute_list)

    empty_row = [None for _ in range(len(df.columns))]
    for i in range(len(hour_minute_list)):
        hour_minute_list[i] = ':'.join(hour_minute_list[i])
        empty_row[0] = df['Date'][0]
        empty_row[1] = hour_minute_list[i]
        df.loc[len(df)] = empty_row

    # df = df.sort_values(by=['Timestamp'], ascending=True)
    # df.to_csv('test.csv', index=False)
    return df

In [68]:
import pandas as pd
import os
import numpy as np
directory = "C:\\Users\\tengwei.goh\\Desktop\\Sample Data\\KAL_test"
os.chdir(directory)
files_with_errors = []
collated_df = pd.DataFrame(columns=['Date','Timestamp'])
collated_df = collated_df.set_index(['Date','Timestamp'])
date_and_time = ''
date_and_time_column_name_list = ['Date and Time', 'date and time', 'DATE AND TIME',
                                    'Date & Time',  'date & time', 'DATE & TIME',
                                    'Date_Time', 'date_time', 'DATE_TIME',
                                    'Date Time', 'date time', 'DATE TIME'
                                    ]
for root,dirs,files in os.walk(directory):
        for file in files:
            if file.endswith('collated.xlsx'):
                continue
                
            if file.endswith('.xlsx'):
                try:
                    df = pd.read_excel(file, sheet_name=0)
                    # print(df.head(6))
                    df = df.reset_index(drop=True)
                    df.columns = df.iloc[7-2]
                    df = df.drop(df.index[:7-1])
                    df = df.reset_index()
                    df = df.dropna(how='all')
                    # df.drop(columns=['index'], axis=1, inplace=True)

                    try:
                        df.columns.get_loc('Timestamp')
                    except:
                        df = df.rename(columns={'Time': 'Timestamp'})

                    for date_time in date_and_time_column_name_list:
                        try:
                            df[date_time] = df[date_time].str.split(' ')
                            date_and_time = date_time
                            break
                        except:
                            pass
                    # df['date and time'] = df['date and time'].str.split(' ')
                    # df["Date"] = ""
                    # df["Timestamp"] = ""
                    # df['Date'] = df['date and time'][0][0]

                    for i in range(len(df[date_and_time])):
                        try:
                            df['Date'] = df[date_and_time][0][0]
                            df['Timestamp'][i] = df[date_and_time][i][1][1:6]
                            # temp = df['Timestamp'][i].split(':')
                            # hour = str('%02d' % int(temp[0])) + ':'
                            # minute = str('%02d' % int(temp[1]))
                            # df['Timestamp'][i] = hour + minute
                        except:
                            pass
                    # print(df['Date'].head())
                    # print(df['Timestamp'].head())
                    df.drop(columns=date_and_time, axis=1, inplace=True)
                    print(df.head())
                    df = df.set_index(['Date','Timestamp'])
                    collated_df = collated_df.combine_first(df)
                    # collated_df = pd.merge(collated_df, df, how='outer', on=['Date','Timestamp']) 
                except Exception as e:
                    print(e)
                    files_with_errors.append(file)
                    continue
    
collated_df.sort_values(by=['Date','Timestamp'], ascending=True)
print(f'These are the files with errors: {files_with_errors}')
# current_date = get_current_date()
collated_df.to_excel('collated.xlsx', index=False)

C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_6544\833673471.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = df[date_and_time][i][1][1:6]


5  index        Date Timestamp  CHWST  CHWRT  CHWF  CDWST CDWRT  CDWF CH1 KW  \
0      6  2022-08-26     23:59  13.31  15.11 -0.05  27.01  27.2 -0.08      0   
1      7  2022-08-26     23:58  13.31  15.11 -0.05  27.04  27.2 -0.08      0   
2      8  2022-08-26     23:57  13.31   15.1 -0.05  27.03  27.2 -0.08      0   
3      9  2022-08-26     23:56  13.31   15.1 -0.05  27.04  27.2 -0.08      0   
4     10  2022-08-26     23:55   13.3   15.1 -0.05  27.04  27.2 -0.08      0   

5  ... NaN NaN NaN NaN NaN                       NaN  \
0  ... NaN NaN NaN NaN NaN       Fri,2022-08-26CHWST   
1  ... NaN NaN NaN NaN NaN  Saturday,00-01-1900CHWST   
2  ... NaN NaN NaN NaN NaN                       NaN   
3  ... NaN NaN NaN NaN NaN                       NaN   
4  ... NaN NaN NaN NaN NaN                       NaN   

5                       NaN NaN                       NaN  \
0       Fri,2022-08-26CHWRT NaN       Fri,2022-08-26CDWST   
1  Saturday,00-01-1900CHWRT NaN  Saturday,00-01-1900CDWST   

C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_6544\833673471.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = df[date_and_time][i][1][1:6]


5  index        Date Timestamp  CHWST  CHWRT  CHWF  CDWST  CDWRT  CDWF CH1 KW  \
0      6  2022-08-27     00:00  13.32  15.11 -0.05  27.05  27.21 -0.08      0   
1      7  2022-08-27     00:01  13.32  15.11 -0.06  27.05  27.21 -0.08      0   
2      8  2022-08-27     00:02  13.32  15.11 -0.05  27.04  27.21 -0.08      0   
3      9  2022-08-27     00:03  13.32   15.1 -0.06  27.04  27.21 -0.08      0   
4     10  2022-08-27     00:04  13.33  15.12 -0.05  27.05  27.21 -0.08      0   

5  ... NaN NaN NaN NaN NaN                       NaN  \
0  ... NaN NaN NaN NaN NaN       Sat,2022-08-27CHWST   
1  ... NaN NaN NaN NaN NaN  Saturday,00-01-1900CHWST   
2  ... NaN NaN NaN NaN NaN                       NaN   
3  ... NaN NaN NaN NaN NaN                       NaN   
4  ... NaN NaN NaN NaN NaN                       NaN   

5                       NaN NaN                       NaN  \
0       Sat,2022-08-27CHWRT NaN       Sat,2022-08-27CDWST   
1  Saturday,00-01-1900CHWRT NaN  Saturday,00-01-1900CD

In [96]:
import os
import pandas as pd
files_with_errors = []
directory = "C:\\Users\\tengwei.goh\\Desktop\\Sample Data\\e2i_test"
os.chdir(directory)
collated_df = pd.DataFrame(columns=['Date','Timestamp'])
collated_df = collated_df.set_index(['Date','Timestamp'])
# temp_df = pd.DataFrame()
count = 0
cols_to_use = ['Date', 'Timestamp']
# collated_df = pd.DataFrame()
for root,dirs,files in os.walk(directory):
    for file in files:
        count += 1
        # print(file)
        # Ignore collated file to avoid errors
        if file.endswith('collated.csv'):
            continue

        if file.endswith('.csv'):
            # print(file)
            # df = pd.read_csv('CET WEST Greenmark Data - Chiller 42-2022.csv')
            try:
                f = open(file, 'r')
                result = []
                for l in f.readlines():
                    vals = [l for l in l.split('#') if l]
                    index = vals[0]
                    result.append(index)
                ...
                # fill dataframe
                f.close()

                for i in range(len(result)):
                    result[i] = result[i].replace('\n','')
                    # result = result.split(',')

                temp = result[1:]

                for i in range(len(temp)):
                    temp[i] = temp[i].split(',')
                df = pd.DataFrame(temp[1:],columns=temp[0])
                # df.head()
                try:
                    df.columns.get_loc('Timestamp')
                except:
                    df = df.rename(columns={'Time': 'Timestamp'})
                # print(file)
                # print(df.head(5))
                df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')
                print(df.Date)
                for i in range(len(df)):
                    temp = df['Timestamp'][i].split(':')
                    hour = str('%02d' % int(temp[0])) + ':'
                    minute = str('%02d' % int(temp[1]))
                    df['Timestamp'][i] = hour + minute

                #TODO: NEED HELP FROM HERE
                # if count > 1:
                cols_to_use = df.columns.tolist()
                # print(cols_to_use)
                # collated_df.reindex(columns=cols_to_use)
                df = insert_empty_slot_1(df)
                df = df.set_index(['Date','Timestamp'])
                # print(collated_df.head())
                collated_df = collated_df.combine_first(df)
                print(collated_df.columns)
                # try:
                #     collated_df = pd.merge(collated_df,df, on=cols_to_use, how='outer')
                #     print(file, ' used df columns', cols_to_use[:4])
                # except:
                #     collated_df = pd.merge(collated_df, df, on=['Date','Timestamp'], how='outer')
                #     print(file, ' used default columns')
                # print(collated_df.dtypes, df.dtypes)
            except Exception as e:
                print(e)
                files_with_errors.append(file)
                continue
collated_df.sort_values(by=['Date','Timestamp'], inplace=True)
# collated_df.reset_index(drop=True, inplace=True)
print(collated_df.tail().to_string())
# grouped_df = collated_df.groupby(['Date', 'Timestamp']).apply(lambda collated_df: collated_df.sort_values(by=['Date', 'Timestamp'], ascending=False, inplace=True))
# print(grouped_df)
print('Files with errors: ', files_with_errors)
collated_df.to_csv('collated.csv')

0      2022-02-11
1      2022-02-11
2      2022-02-11
3      2022-02-11
4      2022-02-11
          ...    
1433   2022-02-11
1434   2022-02-11
1435   2022-02-11
1436   2022-02-11
1437   2022-02-11
Name: Date, Length: 1438, dtype: datetime64[ns]


C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_6544\3308090058.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = hour + minute


Index(['CHW-Chilled Water Supply Temperature',
       'CHW-Chilled Water Return Temperature', 'CHW-Chilled Water Flow Rate',
       'CNW-Chilled Water Supply Temperature',
       'CNW-Chilled Water Return Temperature', 'CNW-Chilled Water Flow Rate'],
      dtype='object')
0      2022-03-01
1      2022-03-01
2      2022-03-01
3      2022-03-01
4      2022-03-01
          ...    
1434   2022-03-01
1435   2022-03-01
1436   2022-03-01
1437   2022-03-01
1438   2022-03-01
Name: Date, Length: 1439, dtype: datetime64[ns]


C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_6544\3308090058.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = hour + minute


Index(['CHW-Chilled Water Supply Temperature',
       'CHW-Chilled Water Return Temperature', 'CHW-Chilled Water Flow Rate',
       'CNW-Chilled Water Supply Temperature',
       'CNW-Chilled Water Return Temperature', 'CNW-Chilled Water Flow Rate'],
      dtype='object')
0      2022-04-01
1      2022-04-01
2      2022-04-01
3      2022-04-01
4      2022-04-01
          ...    
1433   2022-04-01
1434   2022-04-01
1435   2022-04-01
1436   2022-04-01
1437   2022-04-01
Name: Date, Length: 1438, dtype: datetime64[ns]


...

Index(['CHW-Chilled Water Flow Rate', 'CHW-Chilled Water Return Temperature',
       'CHW-Chilled Water Supply Temperature', 'CNW-Chilled Water Flow Rate',
       'CNW-Chilled Water Return Temperature',
       'CNW-Chilled Water Supply Temperature', 'Chiller Q1-1', 'Chiller Q1-2',
       'Chiller Q1-3', 'Chiller Q1-4', 'Chiller Q1-5', 'Chiller Q1-6',
       'Chiller Q1-7', 'Chiller Q1-8', 'Chiller Q1-9', 'Chiller Q1-Incoming',
       'Chiller Q2-1', 'Chiller Q2-10', 'Chiller Q2-11', 'Chiller Q2-12',
       'Chiller Q2-2', 'Chiller Q2-3', 'Chiller Q2-4', 'Chiller Q2-5',
       'Chiller Q2-6', 'Chiller Q2-7', 'Chiller Q2-8', 'Chiller Q2-9',
       'Chiller Q2-Incoming'],
      dtype='object')
0      2022-03-01
1      2022-03-01
2      2022-03-01
3      2022-03-01
4      2022-03-01
          ...    
1434   2022-03-01
1435   2022-03-01
1436   2022-03-01
1437   2022-03-01
1438   2022-03-01
Name: Date, Length: 1439, dtype: datetime64[ns]


C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_6544\3308090058.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = hour + minute


Index(['CHW-Chilled Water Flow Rate', 'CHW-Chilled Water Return Temperature',
       'CHW-Chilled Water Supply Temperature', 'CNW-Chilled Water Flow Rate',
       'CNW-Chilled Water Return Temperature',
       'CNW-Chilled Water Supply Temperature', 'Chiller Q1-1', 'Chiller Q1-2',
       'Chiller Q1-3', 'Chiller Q1-4', 'Chiller Q1-5', 'Chiller Q1-6',
       'Chiller Q1-7', 'Chiller Q1-8', 'Chiller Q1-9', 'Chiller Q1-Incoming',
       'Chiller Q2-1', 'Chiller Q2-10', 'Chiller Q2-11', 'Chiller Q2-12',
       'Chiller Q2-2', 'Chiller Q2-3', 'Chiller Q2-4', 'Chiller Q2-5',
       'Chiller Q2-6', 'Chiller Q2-7', 'Chiller Q2-8', 'Chiller Q2-9',
       'Chiller Q2-Incoming'],
      dtype='object')
0      2022-04-01
1      2022-04-01
2      2022-04-01
3      2022-04-01
4      2022-04-01
          ...    
1433   2022-04-01
1434   2022-04-01
1435   2022-04-01
1436   2022-04-01
1437   2022-04-01
Name: Date, Length: 1438, dtype: datetime64[ns]


C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_6544\3308090058.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = hour + minute


Index(['CHW-Chilled Water Flow Rate', 'CHW-Chilled Water Return Temperature',
       'CHW-Chilled Water Supply Temperature', 'CNW-Chilled Water Flow Rate',
       'CNW-Chilled Water Return Temperature',
       'CNW-Chilled Water Supply Temperature', 'Chiller Q1-1', 'Chiller Q1-2',
       'Chiller Q1-3', 'Chiller Q1-4', 'Chiller Q1-5', 'Chiller Q1-6',
       'Chiller Q1-7', 'Chiller Q1-8', 'Chiller Q1-9', 'Chiller Q1-Incoming',
       'Chiller Q2-1', 'Chiller Q2-10', 'Chiller Q2-11', 'Chiller Q2-12',
       'Chiller Q2-2', 'Chiller Q2-3', 'Chiller Q2-4', 'Chiller Q2-5',
       'Chiller Q2-6', 'Chiller Q2-7', 'Chiller Q2-8', 'Chiller Q2-9',
       'Chiller Q2-Incoming'],
      dtype='object')
                     CHW-Chilled Water Flow Rate CHW-Chilled Water Return Temperature CHW-Chilled Water Supply Temperature CNW-Chilled Water Flow Rate CNW-Chilled Water Return Temperature CNW-Chilled Water Supply Temperature Chiller Q1-1 Chiller Q1-2 Chiller Q1-3 Chiller Q1-4 Chiller Q1-5 Chiller Q1

In [83]:
import os
import pandas as pd
files_with_errors = []
directory = "C:\\Users\\tengwei.goh\\Desktop\\Sample Data\\e2i_test"
os.chdir(directory)
collated_df = pd.DataFrame(columns=['Date','Timestamp'])
# temp_df = pd.DataFrame()
count = 0
cols_to_use = ['Date', 'Timestamp']
# collated_df = pd.DataFrame()
for root,dirs,files in os.walk(directory):
    for file in files:
        count += 1
        # print(file)
        # Ignore collated file to avoid errors
        if file.endswith('collated.csv'):
            continue

        if file.endswith('.csv'):
            print(file)
            # df = pd.read_csv('CET WEST Greenmark Data - Chiller 42-2022.csv')
            f = open(file, 'r')
            result = []
            for l in f.readlines():
                vals = [l for l in l.split('#') if l]
                index = vals[0]
                result.append(index)
            ...
            # fill dataframe
            f.close()

            for i in range(len(result)):
                result[i] = result[i].replace('\n','')
                # result = result.split(',')

            temp = result[1:]

            for i in range(len(temp)):
                temp[i] = temp[i].split(',')
            df = pd.DataFrame(temp[1:],columns=temp[0])
            # df.head()
            try:
                df.columns.get_loc('Timestamp')
            except:
                df = df.rename(columns={'Time': 'Timestamp'})
            # print(file)
            # print(df.head(5))
            for i in range(len(df)):
                temp = df['Timestamp'][i].split(':')
                hour = str('%02d' % int(temp[0])) + ':'
                minute = str('%02d' % int(temp[1]))
                df['Timestamp'][i] = hour + minute

            #TODO: NEED HELP FROM HERE
            if count > 1:
                # cols_to_use = df.columns.difference(temp_df.columns).tolist()
                cols_to_use = df.columns.tolist()
                # temp_df = pd.DataFrame(columns=cols_to_use)
                # print(temp_df.columns)
                # cols_to_use += temp_cols
                print(cols_to_use)
                collated_df.reindex(columns=cols_to_use)
                # collated_df_cols = collated_df.columns.tolist()
                # for col in cols_to_use:
                #     if col not in collated_df_cols:
                #         collated_df[col] = None
                # pd.concat([collated_df,temp_df])
                # print('COLUMNS:', collated_df.columns)
            # collated_df = pd.merge(collated_df, df[cols_to_use], left_index=True, right_index=True, how='outer')
            # print(collated_df.columns)
                collated_df.Date = collated_df.Date.astype(str)
                collated_df.Timestamp = collated_df.Timestamp.astype(str)
                df.Date = df.Date.astype(str)
                df.Timestamp = df.Timestamp.astype(str)
                # collated_df.Date = collated_df.Date.str.encode('utf-8')
                # collated_df.Timestamp = collated_df.Timestamp.str.encode('utf-8')
                # df.Date = df.Date.str.encode('utf-8')
                # df.Timestamp = df.Timestamp.str.encode('utf-8')
                # collated_df.Date = collated_df.Date.str.strip()
                # collated_df.Timestamp = collated_df.Timestamp.str.strip()
                # df.Date = df.Date.str.strip()
                # df.Timestamp = df.Timestamp.str.strip()
                # print(collated_df.Date, collated_df.Timestamp)
                # print(df.Date, df.Timestamp)
            try:
                collated_df = pd.merge(collated_df, df, on=cols_to_use, how='outer', suffixes=('', '_test'))
            except:
                collated_df = pd.merge(collated_df, df, on=['Date','Timestamp'], how='outer' )
            # to_drop = [x for x in collated_df if x.endswith('_test')]
            # # print(to_drop)
            # collated_df.drop(to_drop, axis=1, inplace=True)
print(collated_df.tail())
# collated_df.Date = collated_df.Date.str.encode('utf-8')
# collated_df.Timestamp = collated_df.Timestamp.str.encode('utf-8')
# print(collated_df.Date, collated_df.Timestamp)
collated_df.to_csv('collated.csv', index=False)

CET WEST Greenmark Data - Chiller 41-2022.csv
CET WEST Greenmark Data - Chiller 42-2022.csv
['Date', 'Timestamp', 'CHW-Chilled Water Supply Temperature', 'CHW-Chilled Water Return Temperature', 'CHW-Chilled Water Flow Rate', 'CNW-Chilled Water Supply Temperature', 'CNW-Chilled Water Return Temperature', 'CNW-Chilled Water Flow Rate']
CET WEST Greenmark Data - Chiller 43-2022.csv
['Date', 'Timestamp', 'CHW-Chilled Water Supply Temperature', 'CHW-Chilled Water Return Temperature', 'CHW-Chilled Water Flow Rate', 'CNW-Chilled Water Supply Temperature', 'CNW-Chilled Water Return Temperature', 'CNW-Chilled Water Flow Rate']
CET WEST Greenmark Data - Electrical 41-2022.csv
['Date', 'Timestamp', 'Chiller Q1-Incoming', 'Chiller Q1-1', 'Chiller Q1-2', 'Chiller Q1-3', 'Chiller Q1-4', 'Chiller Q1-5', 'Chiller Q1-6', 'Chiller Q1-7', 'Chiller Q1-8', 'Chiller Q1-9', 'Chiller Q2-Incoming', 'Chiller Q2-1', 'Chiller Q2-2', 'Chiller Q2-3', 'Chiller Q2-4', 'Chiller Q2-5', 'Chiller Q2-6', 'Chiller Q2-7', '

In [7]:
import pandas as pd
import numpy as np

excel_column_header_row = 7
df = pd.read_excel('GMreport_20220827-080004.xlsx', sheet_name=0)
# df = pd.read_excel(file, sheet_name=0)
# print(df.head(6))
df = df.reset_index(drop=True)
df.columns = df.iloc[int(excel_column_header_row)-2]
df = df.drop(df.index[:int(excel_column_header_row)-1])
df = df.reset_index()
df = df.dropna(how='all')
df.drop(columns=['index'], axis=1, inplace=True)

try:
    df.columns.get_loc('Timestamp')
except:
    df = df.rename(columns={'Time': 'Timestamp'})

df['date and time'] = df['date and time'].str.split(' ')
# df["Date"] = ""
# df["Timestamp"] = ""
# df['Date'] = df['date and time'][0][0]

for i in range(len(df['date and time'])):
    try:
        df['Date'] = df['date and time'][0][0]
        df['Timestamp'][i] = df['date and time'][i][1][1:6]
    except:
        pass
print(df['Date'].head())
print(df['Timestamp'].head())

df = df.sort_values(by=['Timestamp'])

cols_to_move = ['Date','Timestamp']
df = df[ cols_to_move + [ col for col in df.columns if col not in cols_to_move ] ]
df.drop(columns=['date and time'], axis=1, inplace=True)
df['Timestamp'].replace('', np.nan, inplace=True)
df.dropna(subset=['Timestamp'], inplace=True)
# df = insert_empty_slot_1(df)
# collated_df = pd.concat([collated_df, df], axis=0)
df.to_excel('test.xlsx', index=False)

C:\Users\tengwei.goh\AppData\Local\Temp\ipykernel_1184\2731157576.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'][i] = df['date and time'][i][1][1:6]


0    2022-08-26
1    2022-08-26
2    2022-08-26
3    2022-08-26
4    2022-08-26
Name: Date, dtype: object
0    23:59
1    23:58
2    23:57
3    23:56
4    23:55
Name: Timestamp, dtype: object


In [6]:
import pandas as pd
# e2i
df = pd.read_csv('CET WEST Greenmark Data - Chiller 42-2022.csv')
df.iloc[1] 
# df.iloc[0].name # unable to reference the second row
# df.loc['Date'].index
# df.iloc[:6]

CET WEST Greenmark Data - Chiller    12.53
Name: (02/04/22, 0:1:00:, 7.06, 8.78, 16.14, 25.98, 29.11), dtype: object

In [10]:
df = pd.read_csv('CET WEST Greenmark Data - Chiller 217-2022.csv')
df.iloc[0] # row 1, but actual row is 2
df.head(0)

,Date,Time,CHW-Chilled Water Supply Temperature,CHW-Chilled Water Return Temperature,CHW-Chilled Water Flow Rate,CNW-Chilled Water Supply Temperature,CNW-Chilled Water Return Temperature,CNW-Chilled Water Flow Rate


: 

In [4]:
# KAL
df = pd.read_excel('GMreport_20220827-080004.xlsx', sheet_name=0)
df.iloc[:6] # actual row 7, but row 6 according to pd.read_excel

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,date and time,Date,Time,CHWST,CHWRT,CHWF,CDWST,CDWRT,CDWF,CH1 KW,...,CDW DELTA,CHW FLOW GPM,CDW FLOW GPM,CHW GPM/RT,CDW GPM/RT,CHILLER KW/RT,CHWP KW/RT,CDWP KW/RT,CT KW/RT,BIN RANGE
